# EAE vs external spleen

In [1]:
# this script do dimension reduce first, then transform new data with the same reducer

In [2]:
import warnings
warnings.filterwarnings("ignore")
import muon as mu
import numpy as np
import scanpy as sc
import scirpy as ir
import pandas as pd
from cycler import cycler
from matplotlib import cm as mpl_cm
from matplotlib import pyplot as plt
from mudata import MuData
sc.set_figure_params(figsize=(4, 4))
sc.settings.verbosity = 2  # verbosity: errors (0), warnings (1), info (2), hints (3)

ModuleNotFoundError: No module named 'muon'

### load EAE data

In [ ]:
%cd /ihome/ylee/yiz133/Code/data_EAE/anndata/

In [ ]:
## read raw data
# adata_gex = sc.read("all_gex.h5ad")
# adata_gex.var_names

In [ ]:
mdata = mu.read("all_Common.h5mu")
mdata

In [ ]:
mdata['gex'].obs['manual_cell_type'] = mdata.obs['manual_cell_type']
adata_gex = mdata['gex']

adata_gex = adata_gex[adata_gex.obs["date"].isin(["0605"])]
# adata_gex = adata_gex[adata_gex.obs["manual_cell_type"].isin(["CD4+ T"])]

adata_CNS = adata_gex[adata_gex.obs['tissue'].isin(["CN"])]
adata_SPL = adata_gex[adata_gex.obs['tissue'].isin(["SP"])]

adata_gex.obs['tissue']

In [ ]:
adata_gex.var_names_make_unique()

adata_gex.var["mt"] = adata_gex.var_names.str.startswith("mt-")
# ribosomal genes
adata_gex.var["ribo"] = adata_gex.var_names.str.startswith(("Rps", "Rpl"))
# hemoglobin genes
adata_gex.var["hb"] = adata_gex.var_names.str.contains("^Hb[^(p)]")

sc.pp.calculate_qc_metrics(
    adata_gex, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
# adata = adata_gex
sc.pp.filter_cells(adata_gex, min_genes= 200)
sc.pp.filter_cells(adata_gex, max_genes= 8000)

sc.pp.filter_cells(adata_gex, min_counts= 100)
sc.pp.filter_cells(adata_gex, max_counts= 30000)

sc.pp.filter_genes(adata_gex, min_cells= 10)
# sc.pp.filter_genes(adata, max_cells=100)

# sc.pp.filter_genes(adata, min_counts=50)
# sc.pp.filter_genes(adata, max_counts=100)

In [ ]:
# remove mt- genes
adata_gex = adata_gex[:, ~adata_gex.var_names.str.startswith(('mt', 'CMO'))].copy()

# normalize
sc.pp.normalize_total(adata_gex)
sc.pp.log1p(adata_gex)

sc.pp.highly_variable_genes(adata_gex, n_top_genes=2000)
adata_gex = adata_gex[:, adata_gex.var['highly_variable']].copy()
adata_gex

## load multiple .mtx sets

In [ ]:
# Reindex anndata2 to match anndata1 genes (fill missing genes with zeros)
import anndata as ad
def merge_anndata_to_base(anndata1, anndata2):
    c_gene = anndata2.var_names.intersection(anndata1.var_names)
    print(len(c_gene))
    missing_genes = anndata1.var_names.difference(anndata2.var_names)

    # Pad missing genes with zeros (assumes dense arrays, can be adapted for sparse)
    if len(missing_genes) > 0:
        import pandas as pd
        import scipy.sparse

        shape = (anndata2.n_obs, len(missing_genes))
        X_pad = np.zeros(shape, dtype=anndata2.X.dtype)
        X_pad = scipy.sparse.csr_matrix(X_pad) if isinstance(anndata2.X, scipy.sparse.spmatrix) else X_pad

        # Create dummy .var for missing genes
        var_pad = pd.DataFrame(index=missing_genes)

        # Create a temporary AnnData with padded genes
        adata_pad = ad.AnnData(X=X_pad, obs=anndata2.obs.copy(), var=var_pad)

        # Add missing genes and reorder to match anndata1
        anndata2_full = ad.concat([anndata2, adata_pad], axis=1)
        anndata2_full = anndata2_full[:, anndata1.var_names]

    else:
        anndata2_full = anndata2[:, anndata1.var_names]
    return anndata2_full

In [ ]:
path = ["/ihome/ylee/yiz133/Code/Data processing/data/External/Spleen GSE256414/1/",
       "/ihome/ylee/yiz133/Code/Data processing/data/External/Spleen GSE256414/2/",
        "/ihome/ylee/yiz133/Code/Data processing/data/External/EAE CNS SPL GSE156718/",
        "/ihome/ylee/yiz133/Code/Data processing/data/External/CNS GSE216390/GSM6671889/",
       ]

file_prefix = ["GSM8098198_Aged-CTRL-1-3_S3.",
              "GSM8098204_Young-CTRL-1-1_S1.",
               "GSE156718_GEX-HTO_5_iLN_",
               "",
              ]

tissues = ['SPL_ext_1', 'SPL_ext_2', 'EAE_ext', "CNS_ext_1"]

In [ ]:
# for i in range(len(path)):
gene_sets = {}
for i in [2,3]:    
    adata_ext = sc.read_10x_mtx(path[i], var_names='gene_symbols', 
                              make_unique=True, cache=False, gex_only=True,
                              prefix = file_prefix[i])
    print(adata_ext)
    
    adata_ext.var_names_make_unique()
    sc.pp.normalize_total(adata_ext)
    sc.pp.log1p(adata_ext)
    adata_ext = adata_ext[:, ~adata_ext.var_names.str.startswith('mt')].copy()
    
    adata = adata_gex
    sc.pp.filter_cells(adata_ext, min_genes= 10)
    sc.pp.filter_cells(adata_ext, max_genes= 8000)
    sc.pp.filter_cells(adata_ext, min_counts=100)
    sc.pp.filter_cells(adata_ext, max_counts=30000)
    sc.pp.filter_genes(adata_ext, min_cells= 10)

    gene_sets[f'adata_ext_{i}'] = merge_anndata_to_base(adata_gex, adata_ext)
    
    gene_sets[f'adata_ext_{i}'].obs['tissue'] = tissues[i]

In [ ]:
adata_gex.var_names.intersection(adata_ext.var_names)

In [ ]:
!pwd

### Merge with EAE data

In [ ]:
all_adatas = {"main": adata_gex, **gene_sets}

In [ ]:
# only training data
# adata_combine = adata_gex

# training and test dict
adata_combine = ad.concat(all_adatas, join="outer")
adata_combine

In [ ]:
# adata_combine = ad.concat([adata_gex, gene_sets['adata_ext_0'], gene_sets['adata_ext_1'], 
#                           gene_sets['adata_ext_2'] ], join="outer")

# adata_combine = ad.concat([adata_gex, gene_sets['adata_ext_3']], join="outer")

In [ ]:
# Concatenate along observations
# adata_combine = ad.concat([anndata1, anndata2_full], axis=0 , join="inner")
# adata_combine.obs['tissue'].value_counts()
# adata_combine

## clusters

In [ ]:
adata_combine.obs['tissue'].value_counts()

In [ ]:
sc.pp.pca(adata_combine, svd_solver="arpack", n_comps=20)

In [ ]:
sc.pp.neighbors(adata_combine, n_neighbors = 20)
sc.tl.umap(adata_combine, min_dist=0.5, spread=1.0)
sc.tl.leiden(adata_combine, resolution = 0.5, n_iterations=2, flavor = 'igraph')
sc.pl.umap(adata_combine, color=["tissue"])

In [ ]:
# DO Batch correction

sc.external.pp.harmony_integrate(adata_combine, "tissue")

sc.pp.neighbors(adata_combine, use_rep='X_pca_harmony')

In [ ]:
sc.tl.umap(adata_combine)
sc.pl.umap(adata_combine, color=["tissue"])

## PCA on the training set

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_idx = adata_combine.obs['tissue'].isin(["CN","SP"])
adata_train = adata_combine[train_idx]
adata_test = adata_combine[~train_idx]

In [ ]:
adata_train

In [ ]:
adata_test

In [ ]:
# external_spl, 
# mdata = mu.read("0605_mData_allCommon.h5mu")
# mdata['gex'].obs = mdata.obs
# adata_gex = mdata['gex']
# adata_gex = adata_gex[adata_gex.obs["manual_cell_type"].isin(["CD4+ T"])]

DF_train = adata_train.to_df()
DF_all = adata_combine.to_df()

In [ ]:
%cd "/ihome/ylee/yiz133/Code/Data processing/"

In [ ]:
# Initialize StandardScaler
import pandas as pd

scaler = StandardScaler()
# scaler =  MinMaxScaler()

# # Fit and transform the data
# # X_scaled will be a NumPy array
# X_scaled_train = scaler.fit_transform(DF_train)
# X_scaled_all = scaler.transform(DF_all)

X_scaled_train = DF_train
X_scaled_all = DF_all

In [ ]:
pca = PCA(n_components=50, svd_solver='randomized', random_state=42)
pca.fit(X_scaled_train)

X_pca_train = pca.transform(X_scaled_train)
X_pca_all = pca.transform(X_scaled_all)

# Convert the transformed data to a DataFrame for easier handling
df_pca_all = pd.DataFrame(data=X_pca_all)

df_pca_all.index = DF_all.index

In [ ]:
idx = len(X_scaled_train)
X_pca_all_sub = X_pca_all[:idx]
pca_diff = X_pca_train - X_pca_all_sub
pca_diff.max()

In [ ]:
DF_classes = adata_combine.obs
DF_classes.index = DF_all.index
DF_classes.head(3)

In [ ]:
import matplotlib.pyplot as plt

# Plot the first two PCA components
fig, ax = plt.subplots(2,2, figsize=(8, 8))

y = DF_classes['tissue'].unique()
i = 0

for category in y:
    X_pca_sub = df_pca_all[DF_classes['tissue'] == category]
    ax[int(i/2), i%2].scatter(
        X_pca_sub.iloc[:,0], X_pca_sub.iloc[:, 1],
        label=category,
        alpha=0.8,
        linewidths=0.0001,
        marker='.'
    )
    ax[int(i/2), i%2].set_xlim(-10, 20)
    ax[int(i/2), i%2].set_ylim(-10, 20)
    i += 1


# plt.title("UMAP")
# plt.legend(title ='tissue')
# plt.show(block=False)


for category in y:
    X_pca_sub = df_pca_all[DF_classes['tissue'] == category]

    # print(X_pca_sub.shape)
    ax[1,1].scatter(
        X_pca_sub.iloc[:,0], X_pca_sub.iloc[:, 1],
        label=category,
        alpha=0.8,
        linewidths=0.0001,
        marker='.'
    )
    plt.xlim(-10, 20)
    plt.ylim(-10, 20)
    
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("PCA - First Two Components")
plt.legend(title="Tissue", loc='right')  # Add legend with title
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8, 8))

y = DF_classes['tissue'].unique()
pca_num = 4

for i in range(pca_num):
    for category in y:
        X_pca_sub = df_pca_all[DF_classes['tissue'] == category]

        ax[int(i/2), i%2].scatter(
            X_pca_sub.iloc[:,0], X_pca_sub.iloc[:, i+1],
            label=category,
            alpha=0.8,
            linewidths=0.0001,
            marker='.'
        )
    
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("PCA - First Two Components")
plt.legend(title="Tissue")  # Add legend with title
plt.grid(True)
plt.tight_layout()
plt.show()

### umap

In [ ]:
import umap

umap = umap.UMAP(n_components=2, n_neighbors=20, min_dist=0.5, spread = 2)
X_umap_all = umap.fit_transform(df_pca_all)

# Convert the transformed data to a DataFrame for easier handling
df_umap_all = pd.DataFrame(data=X_umap_all)

df_umap_all.index = DF_all.index

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(12,12))
i = 0

for category in y:
    X_pca_sub = df_umap_all[DF_classes['tissue'] == category]
    
    ax[int(i/2), i%2].scatter(
        X_pca_sub.iloc[:,0], X_pca_sub.iloc[:, 1],
        label=category,
        alpha=0.8,
        linewidths=0.0001,
        marker='.'
    )
    i += 1

for category in y:
    X_pca_sub = df_umap_all[DF_classes['tissue'] == category]
    ax[1,1].scatter(
        X_pca_sub.iloc[:,0], X_pca_sub.iloc[:, 1],
        label=category,
        alpha=0.8,
        linewidths=0.0001,
        marker='.'
    )
        
plt.title("UMAP")
plt.legend(title ='tissue')
plt.show(block=False)

In [ ]:
%cd "/ihome/ylee/yiz133/Code/Data processing/"

In [ ]:
df_pca_all.to_csv('features_PCA50.csv')
DF_classes.to_csv('obs_classes.csv')
# df_umap_all.to_csv('features_Umap2.csv')


In [ ]:
aaa

## load Spleen GSE134157

In [ ]:
### only external data set can be predicted

In [ ]:
%cd "/ihome/ylee/yiz133/Code/Data processing/data/External/Spleen GSE134157 one set"

In [ ]:
import pandas as pd
EAE_feature = pd.read_csv('EAE_features.tsv', sep='\t', header=None, usecols=[0, 1])
data_dict = dict(zip(EAE_feature[0], EAE_feature[1]))
# data_dict

In [ ]:
external_spl = pd.read_csv('GSE134157_UMImatrix_NICD_protector.tsv', sep='\t', header=0)
external_spl = external_spl.set_index('Unnamed: 0')
external_spl = external_spl.T
external_spl.head(5)

### change gene id to names

In [ ]:
external_spl.columns = external_spl.columns.to_series().map(
    lambda x: data_dict.get(x,x)
)
external_spl.head(5)

In [ ]:
ans = external_spl.columns.to_series().str.startswith('ENSMUSG')
ans.value_counts()

In [ ]:
external_spl.columns.intersection(adata_SPL.var_names)

In [ ]:
import anndata as ad
adata_ext = ad.AnnData(X=external_spl)
# obs=pd.DataFrame(external_spl.index), var=pd.DataFrame(external_spl.columns)
adata_ext = adata_ext[:-2]

In [ ]:
adata_ext.obs['tissue'] = 'SPL_ext'
adata_ext.obs['tissue'] = adata_ext.obs['tissue'].astype('category')
adata_ext.obs['tissue']

In [ ]:
adata_combine = ad.concat((adata_gex,adata_ext), join="outer")

# end

In [ ]:
%pwd

In [ ]:
pca.fit(X_scaled_train)

X_pca_train = pca.transform(X_scaled_train)

# Convert the transformed data to a DataFrame for easier handling
df_pca_2 = pd.DataFrame(data=X_pca_train)
df_pca_2.index = DF_train.index
df_pca_2.to_csv('features_PCA50_train.csv')
